In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

driver.get("https://www.imdb.com/chart/top/")

title = driver.title

driver.implicitly_wait(0.5)


container = driver.find_element(by=By.XPATH, value="//*[@id='__next']/main/div/div[3]/section/div/div[2]/div/ul")

movies = container.find_elements(by=By.CLASS_NAME, value='ipc-metadata-list-summary-item.sc-4929eaf6-0.DLYcv.cli-parent')


data_dicts = []
for movie in movies:
    # Obtener título
    title = movie.find_element(by=By.CLASS_NAME, value='ipc-title-link-wrapper')
    clean_title = title.text.split(' ', 1)[1].upper()

    # Obtener año
    year = movie.find_element(by=By.XPATH, value='.//div[@class="sc-300a8231-6 dBUjvq cli-title-metadata"]//span[1]')


    # Obtener duración
    duration = movie.find_element(by=By.XPATH, value='.//div[@class="sc-300a8231-6 dBUjvq cli-title-metadata"]//span[2]')

    # Obtener rating
    rating = movie.find_element(by=By.CLASS_NAME, value='ipc-rating-star--rating')

    # Agrupar los datos en un solo diccionario
    movie_data = {
        'TITLE': clean_title,
        'YEAR': year.text,
        'DURATION': duration.text,
        'RATING': rating.text
    }

    # Agregar el diccionario a la lista
    data_dicts.append(movie_data)
driver.quit()

# Imprimir el resultado

In [ ]:
import pandas as pd


df = pd.DataFrame(data_dicts)
df.to_excel('top_250_movies.xlsx', index=False)




In [ ]:
df_excel = pd.read_excel('top_250_movies.xlsx')
df_excel

In [4]:
def convert_duration(duration):
    hours, minutes = 0, 0
    if 'h' in duration:
        hours = int(duration.split(' ')[0].replace('h', ''))
        
    if 'm' in duration:
        minutes = int(duration.split(' ')[-1].replace('m', ''))
    return hours * 60 + minutes

In [ ]:
res = convert_duration('2h 7m')
print(res)

In [ ]:
# aplicar conversión
df['DURATION'] = df['DURATION'].apply(convert_duration)
df['DURATION']

In [7]:
def convert_rating(rating):
    return float(rating.replace(',', '.'))

In [ ]:
df['RATING'] = df['RATING'].apply(convert_rating)
df['RATING']

In [ ]:
#  shortest and longest movie

long_movie = df.loc[df['DURATION'].idxmax()]
shortest_movie = df.loc[df['DURATION'].idxmin()]

print(long_movie)
print('\n')
print(shortest_movie)


In [ ]:
hight_rating = df[df['RATING'] > 8.5]
print(hight_rating.head(10))
hight_rating.info()


In [ ]:
# Agrupar por año y calcular estadísticas
estadisticas = df.groupby('YEAR').agg(
    num_peliculas=('TITLE', 'count'),
    # duracion_total=('DURATION', 'sum'),
    # duracion_promedio=('DURATION', 'mean'),
    calificacion_promedio=('RATING', 'mean')
).reset_index()

# Mostrar el resultado
estadisticas_sorted = estadisticas.sort_values(by='YEAR', ascending=False)
estadisticas_sorted

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
#  visualizar la peliculas por año
plt.figure(figsize=(20, 6))
plt.bar(estadisticas_sorted['YEAR'].astype(str), estadisticas_sorted['num_peliculas'], color='skyblue')
plt.xlabel('Año')
plt.ylabel('Número de Películas')
plt.title('Número de Películas por Año')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
#  visualizar la calificación promedio por año
plt.figure(figsize=(20, 6))
plt.plot(estadisticas_sorted['YEAR'].astype(str), estadisticas_sorted['calificacion_promedio'], marker='o', color='green')
plt.xlabel('Año')
plt.ylabel('Calificación Promedio')
plt.title('Calificación Promedio por Año')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
